In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit

In [3]:
vims_wave = np.loadtxt('vims_wave.txt')
new_array = ['_0405','_0607','_0809','_1011','_1213','_1415','_1617']
for number in new_array:
    new_string = 'df'+number+' = '+'pd.read_pickle("e_data/data'+number+'.pkl")'
    exec(new_string)

In [4]:
def powerlaw(x,a,b):
    return a*np.power(x,b)

def gaussian(x,a,mu,sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

band_channels = [29,30,31,32,33,34,46,47,48,49,50,51,52,53,54,55,56,57,58,59,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,102,103,104,105]

def fit_line(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_fit = powerlaw(vims_wave,*my_fit)
    return final_fit

def custom_fit(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_spectra = spectra-powerlaw(vims_wave,*my_fit)
    return final_spectra


In [10]:
spectra_list = []
for number in new_array:
    new_string = 'spectra'+number+' = '+'custom_fit(df'+number+'["spectrum"])'
    exec(new_string)
    spectrum = 'spectra'+number
    new_spectrum = locals()[spectrum]
    spectra_list.append(new_spectrum)

time_list = []
for number in new_array:
    new_string = 'time'+number+' = '+'df'+number+'["ettime"]'
    exec(new_string)
    time = 'time'+number
    new_time = locals()[time]
    time_list.append(new_time)

spec_list = []
for number in new_array:
    new_string = 'spec'+number+' = '+'df'+number+'["spectrum"]'
    exec(new_string)
    spec = 'spec'+number
    new_spec = locals()[spec]
    spec_list.append(new_spec)

In [11]:
for part in time_list:
    print(part.size)

2507
3424
749
1487
723
2490
1275


### Fitting 2004-2005 Spectra

In [15]:
spec_0405

0       [0.0627468, 0.0772501, 0.115962, 0.158278, 0.1...
1       [0.0649121, 0.0794369, 0.119561, 0.161287, 0.1...
2       [0.0686598, 0.0839468, 0.126895, 0.169983, 0.1...
3       [0.0683702, 0.0855701, 0.127764, 0.171474, 0.1...
4       [0.0649137, 0.0816108, 0.120783, 0.164939, 0.1...
                              ...                        
2502    [0.066277, 0.0830038, 0.124997, 0.170421, 0.12...
2503    [0.0661068, 0.0787953, 0.120702, 0.169602, 0.1...
2504    [0.0768355, 0.0907899, 0.137931, 0.188302, 0.1...
2505    [0.076841, 0.0912231, 0.138605, 0.189246, 0.14...
2506    [0.0665543, 0.077868, 0.119514, 0.17071, 0.130...
Name: spectrum, Length: 2507, dtype: object